```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 问题

#h(2em) （二次指派问题）某公司指派n个员工到n个城市工作（每个城市单独一人），希望使所花费的总电话费用尽可能少。n 个员工两两之间每个月通话的时间表示在下面的矩阵的上三角部分（因为通话的时间矩阵是对称的，没有必要写出下三角部分），n 个城市两两之间通话费率表示在下面矩阵的下三角部分（同样道理，因为通话的费率矩阵是对称的，没有必要写出上三角部分）。试求解该二次指派问题（如果你的软件解不了这么大规模的问题，那就只考虑最前面的若干员工和城市）。

= 思路

$
  "target" = (sum_(i, j, m, n = 1)^(i, j, m, n = 10) x_(i j) x_(m n) "time"_(i m) "rate"_(j n)) / 2
$

即求 $"target"$ 最小值。

S.T.

$
  sum_(i = 1)^(i = 10) x_(i j) = 1 \
  sum_(j = 1)^(j = 10) x_(i j) = 1 \
  x_(i j) = 0, 1 \
$

#h(2em) 由于 scipy 的 scipy.optimize.quadratic_assignment 接口能解决的问题有限（scipy 无法解决 0-1 决策问题），尝试调用 scipy.optimize.minimize 作为一般规划问题求解，多次尝试无果后调用 gurobipy 库进行求解。

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
# 导入数据
data = np.array(
  [
    [0, 5, 3, 7, 9, 3, 9, 2, 9, 0],
    [7, 0, 7, 8, 3, 2, 3, 3, 5, 7],
    [4, 8, 0, 9, 3, 5, 3, 3, 9, 3],
    [6, 2, 10, 0, 8, 4, 1, 8, 0, 4],
    [8, 6, 4, 6, 0, 8, 8, 7, 5, 9],
    [8, 5, 4, 6, 6, 0, 4, 8, 0, 3],
    [8, 6, 7, 9, 4, 3, 0, 7, 9, 5],
    [6, 8, 2, 3, 8, 8, 6, 0, 5, 5],
    [0, 3, 6, 2, 8, 3, 7, 8, 0, 5],
    [5, 6, 7, 6, 6, 2, 8, 8, 9, 0],
  ],
  dtype=np.int32,
)

N: int = 10

time = np.triu(data)
time = time + time.T

rate = np.tril(data)
rate = rate + rate.T

In [ ]:
# 定义目标函数
def target(_x: np.ndarray):
  x = _x.reshape(N, N)
  temp = 0
  for i in range(N):
    for j in range(N):
      for m in range(N):
        for n in range(N):
          temp = temp + x[i, j] * x[m, n] * time[i, m] * rate[j, n]
  return -temp


useArr0 = np.zeros(N, dtype=np.int32)
useArr1 = np.ones(N, dtype=np.int32)

nlc0 = opt.NonlinearConstraint(lambda x: np.sum(x.reshape(N, N), axis=0), useArr0, useArr1)
nlc1 = opt.NonlinearConstraint(lambda x: np.sum(x.reshape(N, N), axis=1), useArr0, useArr1)

x0 = np.array(
  [
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  ],
  dtype=np.int32,
).flatten()

res = opt.minimize(target, x0, constraints=[nlc0, nlc1])

print(res)

In [ ]:
import gurobipy as gp

model = gp.Model('model')

x = model.addVars(N, N, vtype=gp.GRB.BINARY, name='x')

objective = gp.quicksum(
  x[i, j] * x[m, n] * time[i, m] * rate[j, n] for i in range(N) for j in range(N) for m in range(N) for n in range(N)
)

model.setObjective(objective / 2, gp.GRB.MINIMIZE)

model.addConstrs((x.sum(i, '*') == 1 for i in range(N)), name='row')
model.addConstrs((x.sum('*', j) == 1 for j in range(N)), name='col')

model.optimize()

if model.status == gp.GRB.OPTIMAL:
  for i in range(N):
    for j in range(N):
      if x[i, j].x > 0.5:
        print(f'{i} -> {j}')
        pass
      pass
    pass
  pass

指派结果如下（前面为人，后面为前往的城市）：

```txt
0 -> 8
1 -> 0
2 -> 7
3 -> 2
4 -> 5
5 -> 6
6 -> 1
7 -> 4
8 -> 3
9 -> 9
```
